In [1]:
import sys 
sys.path.append('../../src/anomaly_detection_spatial_temporal_data/')

In [18]:
import pandas as pd
import numpy as np
import yaml
from model.GDN.GDNTrainer import GDNTrainer

# Load data

### Load sensor columns

In [3]:
data_dir = "../../data/03_primary"

In [4]:
with open(f"{data_dir}/iot_sensor_list_batadal.txt", "r") as f:
    sensors = f.read().split("\n")

In [5]:
print(f"Number of sensors: {len(sensors)}")

Number of sensors: 43


In [6]:
print(sensors)

['L_T1', 'L_T2', 'L_T3', 'L_T4', 'L_T5', 'L_T6', 'L_T7', 'F_PU1', 'S_PU1', 'F_PU2', 'S_PU2', 'F_PU3', 'S_PU3', 'F_PU4', 'S_PU4', 'F_PU5', 'S_PU5', 'F_PU6', 'S_PU6', 'F_PU7', 'S_PU7', 'F_PU8', 'S_PU8', 'F_PU9', 'S_PU9', 'F_PU10', 'S_PU10', 'F_PU11', 'S_PU11', 'F_V2', 'S_V2', 'P_J280', 'P_J269', 'P_J300', 'P_J256', 'P_J289', 'P_J415', 'P_J302', 'P_J306', 'P_J307', 'P_J317', 'P_J14', 'P_J422']


### Load train and test CSVs

In [7]:
train_df = pd.read_csv(f"{data_dir}/iot_gdn_train.csv")
test_df = pd.read_csv(f"{data_dir}/iot_gdn_test.csv")

print(train_df.shape)
print(test_df.shape)


(8761, 44)
(2089, 45)


### Load training and environment configurations

In [8]:
model_config_file = "../../conf/base/parameters/gdn.yml"

In [9]:
with open(model_config_file, "r") as stream:
    try:
        model_config = yaml.safe_load(stream)
        print(model_config)
    except yaml.YAMLError as exc:
        print(exc)

{'env_config': {'checkpoint_save_dir': 'data/07_model_output/', 'report': 'best', 'device': 'cpu', 'load_model_path': ''}, 'train_config': {'seed': 5, 'batch': 32, 'slide_win': 5, 'dim': 64, 'out_layer_num': 1, 'slide_stride': 1, 'topk': 5, 'out_layer_inter_dim': 128, 'val_ratio': 0.2, 'decay': 0, 'epoch': 3, 'comment': ''}}


In [10]:
train_config = model_config["train_config"]
env_config = model_config["env_config"]

env_config["checkpoint_save_dir"] = "../../data/07_model_output/gdn-iot-notebook"

# Train model

In [11]:
trainer = GDNTrainer(
    sensors, train_df, test_df, 
    train_config, env_config
)

In [12]:
trainer.run()

epoch (0 / 3) (Loss:0.10291515, ACU_loss:22.53841695)
epoch (1 / 3) (Loss:0.05298028, ACU_loss:11.60268057)
epoch (2 / 3) (Loss:0.04689803, ACU_loss:10.27066912)


# Run inference

In [13]:
pred, labels = trainer.predict()

In [15]:
pred.shape

(2084,)

In [19]:
np.array(labels).shape

(2084,)